In [2]:
import os, ffmpy, subprocess, json, dateutil, re, shutil, fnmatch, natsort
from glob import glob
import pandas as pd
import numpy as np

In [5]:
# location of all ir images
datapath = '/media/sanromd/data/lab/data/laser_experiments/_unsorted/irdata/laser-experiments/perforation/'
#logpath  = '/media/sanromd/data/lab/data/_logbooks/_flir/logbook_processing.csv'
logpath  = '/media/sanromd/data/lab/data/_logbooks/_flir/perf.csv'

In [20]:
rock_dict = {
    'lssw':{'family':'carbonate', 'genus':'limestone', 'species':'limestone saudi white', 
            'origin':'sau', 'type':'outcrop', 'basecode':'ls'},
    'lssy':{'family':'carbonate', 'genus':'limestone', 'species':'limestone saudi yellow', 
            'origin':'sau', 'type':'outcrop',  'basecode':'ls'},
    'lsu':{'family':'carbonate', 'genus':'limestone', 'species':'limestone unknown origin', 
           'origin':'na', 'type':'outcrop',  'basecode':'ls'},
    'lsi':{'family':'carbonate', 'genus':'limestone', 'species':'limestone indiana',
           'origin':'usa', 'type':'outcrop',  'basecode':'ls'},
    'lsy':{'family':'carbonate', 'genus':'limestone', 'species':'limestone yellow',
           'origin':'usa', 'type':'outcrop',  'basecode':'ls'},
    'dl':{'family':'carbonate', 'genus':'limestone', 'species':'dolomitic limestone', 
          'origin':'usa', 'type':'outcrop',  'basecode':'dl'},
    'cb':{'family':'carbonate', 'genus':'unkown', 'species':'carbonate unkown', 
          'origin':'sau', 'type':'field',  'basecode':'cb'},
    'sh':{'family':'shale', 'genus':'shale', 'species':'shale', 
          'origin':'sau', 'type':'field',  'basecode':'sh'},
    'un':{'family':'unknown', 'genus':'unknown', 'species':'unknown',
          'origin':'sau', 'type':'field',  'basecode':'un'},
}

In [21]:
# folderdict = {'png':'figs', 'txt':'stats','ats':'ats'}

# def filename_renumber(files):
#     # get the filenames only
#     files = [f.split('/')[-1] for f in files]

#     # sort the files
#     files.sort()

#     # determine zfill
#     nfills = len(str(len(files)))

#     # renumerate the files 
#     files_new = ['f_'+str(n).zfill(nfills)+'.'+f.split('.')[-1] for n, f in enumerate(files)]
    
#     return files, files_new

# def sortfiles(datapath):
#     for root, dirs, fs in os.walk(datapath):
#         for p in dirs:
#             if 'all' in p:
#                 pngpath = os.path.join(root,p)
#                 for s in ['figs','stats','ats']:
#                     if not os.path.exists (os.path.join(root,s)):
#                         os.mkdir(os.path.join(root,s))
#                 for f in files:
#                     if f.endswith('ats'):
#                         f_old = os.path.join(root,f)
#                         f_new = os.path.join(root,'ats',f)
#                         shutil.move(f_old, f_new)

#                 for ext in ['png','txt']:
#                     files = glob(os.path.join(pngpath,'*.'+ext))
#                     fsold, fsnew = filename_renumber(files)

#                     for n in range(len(files)):
#                         oldfile = os.path.join(pngpath,fsold[n])
#                         newfile = os.path.join(root,folderdict[ext],fsnew[n])
#                         shutil.move(oldfile, newfile)

In [233]:
class flir:
    out_options = {'c':'copy'}
    
    lowres_params = {'append':'low',
                     'vcodec':'libx264'}
    
    overwrite = False
    testrun = False
    trims   = {}

    _gendict =  {'size':'s',
                 'frame rate':'r',
                 'vcodec':'c:v',
                 'tstart':'ss',
                 'duration':'t',
                 'c':'c',
                 't':'t',
                 'ss':'ss',
                 'c:v':'c:v',
                 'r':'r',
                 's':'s',
                 'append':'',
                 'quality':'c'}
    
    _ignore_keys = ['format', 'append', 'suffix', 'sep']
    
    default_params = {
        'filename':{'append':'',
                    'suffix':'',
                    'sep':'-'
                   },
        'audio':{
            'outformat':'wav'
        },
        'video':{
            'outformat':'mp4'
        }
    }
    
    sep = '-'
    
    fprobe_params = {'print_format':'json',
                     'v':'quiet',
                     'show_entries':'stream=index,codec_type:stream_tags=creation_time:format_tags=creation_time'}
    
    def _filename_basics(self,dictionary):
        out = []
        for key,value in self.default_params['filename'].items():
            if key not in dictionary.keys():
                out.append(value)
            else:
                out.append(dictionary[key])
        
        return out
    
    def _create_outfile(self, basepath, basename, outformat, append='', suffix=''):
        
        if append != '':
            append = append + self.sep
        
        if suffix != '':
            suffix = self.sep + suffix
            
        outfile = os.path.join(basepath,append + basename + suffix + '.' + outformat)
        return outfile
    
    def _unpack_dict_to_list(self, dictionary, key_append='-', val_append='', translate=True):
        ulist = []
        for key, value in dictionary.items():
            if key not in self._ignore_keys:
                if translate:
                    key = self._gendict[key]
                temp = [key_append + key,val_append + value]
                ulist = ulist + temp
        return ulist
    
    def _ff(self,indict, outdict, debug=False, run=True):
        
        if debug:
            print('input ', indict)
            print('output', outdict)
        ff = ffmpy.FFmpeg(inputs=indict, outputs=outdict)
        
        if debug or (not self.testrun):
            print('running command \n', ff.cmd)
        if not self.testrun:
            ff.run()
        return

    inputs  = {
        'r':'0',
        'f':'image2',
        'pattern_type':'glob'
    }

    outputs = {
        'c:v':'libx264',
        'pix_fmt':'yuv420p',
        'preset':'medium',
        'crf':'25'
    }

    @property
    def _zfill(self,path=None, files=None, ext='png'):

        if files is None:
            f_N = len(fnmatch.filter(os.listdir(path), '*.'+ext))
        else:
            f_N   = len(files)

        zfill = int(np.log10(f_N))+1
        return zfill

    def _strfill(self, string, zfill, splitter='/', index=-1):
        ftemp = string.split(splitter)[index]
        fn = re.findall(r'[0-9]+', ftemp)[0].zfill(zfill)
        return fn

    def zfill_filenames(self, path, ext='png', zfill='auto', inplace=False, dstpath=None, dryrun=False, debug=False,
                        append='f_'):

        figfiles = glob(os.path.join(path,'*.'+ext))
        figfiles = natsort.natsorted(figfiles)

        if zfill=='auto':
            zfill = self._zfill(files=figfiles)

        if inplace:
            dstpath = path

        if not os.path.exists(dstpath):
            os.mkdir(dstpath)

        for fname in figfiles:
            fn = self._strfill(fname, zfill)
            oldname = fname
            newname = os.path.join(dstpath,append + fn + '.' + ext)
            if (dryrun or debug):
                print('moving ', oldname, 'to '. newname, sep='\t')
            if not dryrun:
                shutil.move(oldname, newname)
        return

    def timelapse(self,srcpath, dstpath, dstname, suffix='', ext='mp4', dryrun=False, debug=False):
        infile= os.path.join(srcpath,'*.png')
        infile = infile.replace(' ','')

        outfile = os.path.join(dstpath, dstname + suffix + '.' + ext)

        ff_inlist  = self._unpack_dict_to_list(self.inputs, translate=False)
        ff_outlist = self._unpack_dict_to_list(self.outputs, translate=False) + ['-an', '-y']

        ff = ffmpy.FFmpeg(inputs={infile:ff_inlist}, outputs={outfile:ff_outlist})

        # print(np.int(len(figfiles)/row['t_max']), row['f_rate'], row['f_N'], row['presets'], row['tag'],sep='\t')
        if (dryrun or debug):
            print(ff.cmd)
        if not dryrun:
            ff.run()
        return
    
    
#     def timelapse(self,video_path, trims, options,
#                     input_format = None, output_format = None,
#                     append_folder = False, join_cuts = False, 
#                     debug = False, dryrun = False):
#         fpath = video_path
#         optlist = self._unpack_dict_to_list(options, translate=False)
        
#         if join_cuts:
#                 fjoin = open(os.path.join(fpath,'concat.txt'),'w')
#                 foutc = os.path.join(fpath,'cuts-combined')
#                 files = []
#         for key in trims.keys():
#             try:
#                 fbase = key.split('.')[0]
#             except:
#                 fbase = key

#             if input_format is None:
#                 input_format = trims[key]['input_format']

#             if output_format is None:
#                 output_format = trims[key]['output_format']

#             append_filename = ''
#             if 'append' in trims[key]:
#                 append_filename = trims[key]['append']
#             if append_folder:
#                 append_filename = d + '-' + append_filename

#             fname_in  = glob.glob(os.path.join(fpath,fbase+'*.'+input_format))[0]
#             fname_out = os.path.join(fpath,append_filename + '-' + fbase + '.' + output_format)

#             if trims[key]['audio'] is not '':
#                 opts = ['-an'] + optlist

#             trimopts = ['-ss', trims[key]['tstart'],
#                     '-t',trims[key]['duration']] 
#             outopts  = trimopts+ opts

#             if output_format.lower() == 'wmv':
#                 outopts = ['-qscale', '0'] + outopts

#             if (d is not '') and (d in flipvidsinfolders):
#                 outopts = ['-vf', 'transpose=1'] + outopts
#                 print(d,' rotate')

#             indict  = {fname_in:'-y'}
#             outdict = {fname_out:outopts}

#             ff[key] = ffmpy.FFmpeg(inputs=indict, outputs=outdict)
#         for key in ff.keys():
#             print(ff[key].cmd+'\n')
#             print('running ffmpeg...')
#             if not dryrun:
#                 ff[key].run()
#             print('done \n')

In [184]:
colnames = ['tag', 'subtag', 'test_n', 'presets', 'f_N', 't_max', 'f_laseron','f_bkt']
flirlog  = pd.read_csv(logpath, header=1, names=colnames)

# get basecodes and codes for each rock
flirlog['code'] = flirlog['tag'].apply(lambda x: re.findall(r'[a-z]+',x)[0]).astype('category')
flirlog['basecode'] = flirlog['code'].apply(lambda x: rock_dict[x]['basecode']).astype('category')

# set the test_n as folder
flirlog.loc[flirlog['test_n'].isna()==False,'test_n'] = flirlog.loc[
    flirlog['test_n'].isna()==False,'test_n'].apply(lambda x: 'test_'+str(int(x)).zfill(2))

# handle nans
flirlog.replace(np.nan,'', inplace=True)

# reorder dataframe
flirlog = flirlog.reindex(columns=[colnames[0],'code','basecode']+colnames[1:])

# calculate the frame rate based on the presets
flirlog['f_rate'] = np.round((flirlog['f_N'].values/flirlog['presets'].values)/flirlog['t_max'].values,0)
flirlog['f_rate'] = flirlog['f_rate'].astype(np.int)
# load helper functions from class
ft = flir_timelapse()

In [235]:
ft = flir()

for index, row in flirlog.iterrows():
    samplepath = os.path.join(row['tag'], row['subtag'], row['test_n'])
    statpath   = os.path.join(datapath, samplepath, 'stat')
    srcpath = os.path.join(datapath, samplepath, 'sorted_png')
    
    # set frame rate
    ft.inputs['r'] = str(row['f_rate'])
    
    # check vidpath and create it if necessary
    dstpath    = os.path.join(datapath, samplepath, 'vids')
    if not os.path.exists(dstpath):
        os.mkdir(dstpath)
    
    ft.timelapse(srcpath, dstpath, row['tag'], dryrun=True)

ffmpeg -r 147 -f image2 -pattern_type glob -i /media/sanromd/data/lab/data/laser_experiments/_unsorted/irdata/laser-experiments/perforation/cb_003/sorted_png/*.png -c:v libx264 -pix_fmt yuv420p -preset medium -crf 25 -an -y /media/sanromd/data/lab/data/laser_experiments/_unsorted/irdata/laser-experiments/perforation/cb_003/vids/cb_003.mp4
ffmpeg -r 127 -f image2 -pattern_type glob -i /media/sanromd/data/lab/data/laser_experiments/_unsorted/irdata/laser-experiments/perforation/cb_004/sorted_png/*.png -c:v libx264 -pix_fmt yuv420p -preset medium -crf 25 -an -y /media/sanromd/data/lab/data/laser_experiments/_unsorted/irdata/laser-experiments/perforation/cb_004/vids/cb_004.mp4
ffmpeg -r 135 -f image2 -pattern_type glob -i /media/sanromd/data/lab/data/laser_experiments/_unsorted/irdata/laser-experiments/perforation/cb_005/sorted_png/*.png -c:v libx264 -pix_fmt yuv420p -preset medium -crf 25 -an -y /media/sanromd/data/lab/data/laser_experiments/_unsorted/irdata/laser-experiments/perforation/

In [226]:
shutil.copy?

In [229]:
copypath = '/media/sanromd/tmp_data/flir_videos'

for index, row in flirlog.iterrows():
    samplepath = os.path.join(row['tag'], row['subtag'], row['test_n'])
        
    vidpath  = os.path.join(datapath, samplepath, 'vids')
    vidfiles = glob(os.path.join(vidpath,'*.mp4'))
    
    for vfile in vidfiles:
        dstfile = os.path.join(copypath,vfile.split('/')[-1])
        shutil.copy(vfile, dstfile)

In [15]:
# # select only perforation files
# flirlog = flirlog.loc[flirlog['test']=='perforation'].copy()
# flirlog.loc[:,flirlog.columns[1]] = [s.lower() for s in flirlog.loc[:,flirlog.columns[1]].values]
# flirlog.loc[:,'subsample'] = False
# flirlog.loc[:,'test_n'] = 1

# # set not breakthroughs as -1
# flirlog.loc[:,'f_bkt'].replace(np.nan, -1, inplace=True)

# # correct the frame offset
# ix = flirlog.loc[:,'f_bkt']>0
# flirlog.loc[ix,'f_bkt_reset'] = flirlog.loc[ix,'f_bkt'] - flirlog.loc[ix,'f_laseron']
# #flirlog.loc[:,'f_laseron'] = 1

In [19]:
# check if the folders match the data available 
flirlog.loc[:,'path_match'] = False
folders = [f for f in os.listdir(datapath) if not re.match('_', f)]
for f in folders:
    flirlog.loc[flirlog.loc[:,'tag']==f,'path_match'] = True

In [95]:
tag = 'un_005'
true_bkt = 6648
# flirlog.loc[flirlog['tag']==tag,'true_bkt'] = true_bkt
# flirlog.loc[flirlog['tag']==tag,'true_laser_on'] = 3
# flirlog.loc[flirlog['tag']==tag,'test_n'] = ''
flirlog.loc[(flirlog['tag']==tag)&(flirlog['test_n']=='test_01'),'true_bkt'] = true_bkt
flirlog.loc[(flirlog['tag']==tag)&(flirlog['test_n']=='test_01'),'true_laser_on'] = 1

In [186]:
flirlog.to_csv('backup.csv')

In [12]:
# create the figpath and load the images 
ft = flir_timelapse()
inputs  = {
    'r':'0',
    'f':'image2',
}

outputs = {
    'c:v':'libx264',
    'pix_fmt':'yuv420p',
    'preset':'medium',
    'crf':'25'
}


for i,row in flirlog.iterrows():
    if row['path_match']:
        figpath = os.path.join(datapath, row['tag'],row['subsample'], row['test_n'], 'figs')
        vidpath = os.path.join(datapath, row['tag'],row['subsample'], row['test_n'], 'vids')
        if not os.path.exists(vidpath):
            os.mkdir(vidpath)
        # get the number of files (redundacy check)
        f_N = len(fnmatch.filter(os.listdir(figpath), '*.png'))
        f_d = int(np.log10(f_N))+1
        infile= os.path.join(figpath,'f_%0'+str(f_d)+'d.png')
        infile = infile.replace(' ','')
        inputs['r'] = str(np.int(row['f_rate']/row['presets'])) 
        outfile = os.path.join(vidpath,row['tag']+'.mp4')
        if len(os.listdir(vidpath))==0:
            ff_inlist  = ft._unpack_dict_to_list(inputs, translate=False)
            ff_outlist = ft._unpack_dict_to_list(outputs, translate=False) + ['-an', '-y']

            ff = ffmpy.FFmpeg(inputs={infile:ff_inlist}, outputs={outfile:ff_outlist})
            ff.run()

In [22]:
# create the figpath and low the images 
ft = flir_timelapse()
inputs  = {
    'r':'0',
    'f':'image2',
}

outputs = {
    'c:v':'libx264',
    'pix_fmt':'yuv420p',
    'preset':'medium',
    'crf':'25'
}


for i,row in flirlog.iterrows():
    if row['path_match']:
        figpath = os.path.join(datapath, row['tag'],row['subsample'], row['test_n'], 'figs')
        vidpath = os.path.join(datapath, row['tag'],row['subsample'], row['test_n'], 'vids')
        if not os.path.exists(vidpath):
            os.mkdir(vidpath)
        # get the number of files (redundacy check)
        f_N = len(fnmatch.filter(os.listdir(figpath), '*.png'))
        f_d = int(np.log10(f_N))+1
        f_r = np.int(row['f_rate']/row['presets'])
        
        if (row['f_bkt']>0 and row['f_bkt']<=f_N):
            print(f_N, row['f_bkt'], row['tag'],sep='\t')
            
            infile= os.path.join(figpath,'f_%0'+str(f_d)+'d.png')
            
            inputs['start_number'] = '1'
            outputs['vframes'] = str(np.int(row['f_bkt']+5))
            for x in [f_r,np.int(f_r/10.)]:
                inputs['r'] = str(x)
                outfile = os.path.join(vidpath,'_'.join([row['tag'],'bkt',inputs['r']+'x.mp4']))
                ff_inlist  = ft._unpack_dict_to_list(inputs, translate=False)
                ff_outlist = ft._unpack_dict_to_list(outputs, translate=False) + ['-an', '-y']
                ff = ffmpy.FFmpeg(inputs={infile:ff_inlist}, outputs={outfile:ff_outlist})
                ff.run()

2571	2164.0	cb_003
1561	262.0	cb_004
2018	712.0	cb_005
21061	18174.0	cb_006
2571	2210.0	lsi_005
2187	1444.0	lsi_006
6001	5534.0	lssw_002
4571	3448.0	lssw_003
14779	10087.0	lssy_008
7142	7041.0	lssy_011
6001	5494.0	lssy_011
15129	13228.0	lssy_018
2196	1507.0	lsu_004
2100	1240.0	lsy_005
2254	1554.0	sh_013
2174	1410.0	sh_014
3974	1367.0	sh_016
2127	1304.0	sh_017
51143	31132.0	un_002


In [54]:
np.mod(row['f_bkt']+176,f_r)

0.0

In [50]:
(row['f_bkt']+176)/f_r

71.0

In [114]:
fname = os.path.join(datapath,'cb_003','ats','cb_003.ats')
import io
with io.open(fname, 'r') as f:
    for line in f:
        print(line.rstrip('\n\x00'))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf6 in position 32: invalid start byte

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf6 in position 32: invalid start byte

In [102]:
np.load